In [5]:
from pydataset import data
import pyspark
import pyspark.ml
from pyspark.sql.functions import *

import wrangle
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

## Use the .randomSplit method to split the 311 data into training and test sets.

In [7]:
df = wrangle.wrangle_311(spark)

[wrangle.py] reading case.csv
[wrangle.py] handling data types
[wrangle.py] parsing dates
[wrangle.py] adding features
[wrangle.py] joining departments


In [8]:
# split our data into train, test

train, test = df.randomSplit([0.8, 0.2], 19)

In [11]:
def shape(df: pyspark.sql.DataFrame):
    return df.count(), len(df.columns)

In [12]:
shape(train)

(673349, 20)

In [13]:
shape(test)

(168355, 20)

In [15]:
train.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 219                  
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
only showing top 1 row



In [41]:
def spark_df_nan_count(df):
    return df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])

spark_df_nan_count(df).show(vertical=True, truncate=False)

AnalysisException: "cannot resolve 'isnan(`case_opened_date`)' due to data type mismatch: argument 1 requires (double or float) type, however, '`case_opened_date`' is of timestamp type.;;\n'Aggregate [count(CASE WHEN (isnan(cast(case_id#10 as double)) || isnull(case_id#10)) THEN case_id END) AS case_id#4009L, count(CASE WHEN (isnan(case_opened_date#98) || isnull(case_opened_date#98)) THEN case_opened_date END) AS case_opened_date#4011, count(CASE WHEN (isnan(case_closed_date#113) || isnull(case_closed_date#113)) THEN case_closed_date END) AS case_closed_date#4013, count(CASE WHEN (isnan(case_due_date#128) || isnull(case_due_date#128)) THEN case_due_date END) AS case_due_date#4015, count(CASE WHEN (isnan(case_late#68) || isnull(case_late#68)) THEN case_late END) AS case_late#4017, count(CASE WHEN (isnan(num_days_late#15) || isnull(num_days_late#15)) THEN num_days_late END) AS num_days_late#4019L, count(CASE WHEN (isnan(case_closed#53) || isnull(case_closed#53)) THEN case_closed END) AS 

## Create a classification model to predict whether a case will be late or not (i.e. predict case_late). Experiment with different combinations of features and different classification algorithms.

In [19]:
train.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 12:29:00  
 case_due_date        | 2020-09-26 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 005                  
 num_weeks_late       | -142.6441088         
 zipcode              | 78207                
 case_age             | 219                  
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
only showing top 1 row



In [31]:
train.dtypes

[('case_id', 'int'),
 ('case_opened_date', 'timestamp'),
 ('case_closed_date', 'timestamp'),
 ('case_due_date', 'timestamp'),
 ('case_late', 'boolean'),
 ('num_days_late', 'double'),
 ('case_closed', 'boolean'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string'),
 ('num_weeks_late', 'double'),
 ('zipcode', 'string'),
 ('case_age', 'int'),
 ('days_to_closed', 'int'),
 ('case_lifetime', 'int'),
 ('department', 'string'),
 ('dept_subject_to_SLA', 'boolean')]

In [28]:
# Using department and council_district first

In [20]:
rf = pyspark.ml.feature.RFormula(formula='case_late ~ department + council_district').fit(train)
train_input = rf.transform(train)
train_input.show(1)

+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+--------------------+------------------+-----------+---------+--------------------+----------------+--------------------+-------+--------+--------------+-------------+--------------------+-------------------+--------------------+-----+
|   case_id|   case_opened_date|   case_closed_date|      case_due_date|case_late|      num_days_late|case_closed|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district|      num_weeks_late|zipcode|case_age|days_to_closed|case_lifetime|          department|dept_subject_to_SLA|            features|label|
+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+--------------------+------------------+-----------+---------+--------------------+----------------+--------------------+-------+--------+--------------+-------------+-------------------

In [21]:
lr = pyspark.ml.classification.LogisticRegression()

In [22]:
lr_fit = lr.fit(train_input)

In [23]:
lr_fit.summary.areaUnderROC

0.6360295499748985

In [24]:
evaluator = pyspark.ml.evaluation.BinaryClassificationEvaluator()
test_auc = evaluator.evaluate(lr_fit.transform(rf.transform(test)))
test_auc

0.6369172734209141

In [25]:
test_input = rf.transform(test)

In [27]:
(lr_fit.transform(test_input)
 .select('case_late', 'department', 'council_district', 'label', 'probability', 'prediction')
 .groupby('prediction') # predicted == rows
 .pivot('label') # actual values are columns
 .count()
 .show())

+----------+------+-----+
|prediction|   0.0|  1.0|
+----------+------+-----+
|       0.0|149412|18402|
|       1.0|   149|  392|
+----------+------+-----+



In [32]:
rf = pyspark.ml.feature.RFormula(formula='case_late ~ department + source_id').fit(train)
train_input = rf.transform(train)
train_input.show(1)

+----------+-------------------+-------------------+-------------------+---------+------------------+-----------+--------------------+--------+-----------+---------+--------------------+----------------+--------------+-------+--------+--------------+-------------+--------------------+-------------------+--------------------+-----+
|   case_id|   case_opened_date|   case_closed_date|      case_due_date|case_late|     num_days_late|case_closed|service_request_type|SLA_days|case_status|source_id|     request_address|council_district|num_weeks_late|zipcode|case_age|days_to_closed|case_lifetime|          department|dept_subject_to_SLA|            features|label|
+----------+-------------------+-------------------+-------------------+---------+------------------+-----------+--------------------+--------+-----------+---------+--------------------+----------------+--------------+-------+--------+--------------+-------------+--------------------+-------------------+--------------------+-----+
|

In [33]:
lr = pyspark.ml.classification.LogisticRegression()

In [34]:
lr_fit = lr.fit(train_input)

In [35]:
lr_fit.summary.areaUnderROC

0.6368578325642402

In [38]:
evaluator = pyspark.ml.evaluation.BinaryClassificationEvaluator()
# test_auc = evaluator.evaluate(lr_fit.transform(rf.transform(test)))
# test_auc

In [39]:
test_auc = evaluator.evaluate(lr_fit.transform(rf.transform(test)))
test_auc

Py4JJavaError: An error occurred while calling o817.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 238.0 failed 1 times, most recent failure: Lost task 0.0 in stage 238.0 (TID 1282, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$9: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage2.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:636)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at org.apache.spark.util.collection.ExternalSorter.insertAll(ExternalSorter.scala:191)
	at org.apache

## Create a regression model to predict how many days late a case will be (i.e. predict num_days_late). Experiment with different combinations of features and different regression algorithms.